In [1]:
import boto3
import json
import bson
#from bson import json_util

import pandas as pd
pd.options.display.max_colwidth = 999
pd.options.display.max_rows = 300

## Pre-processing

In [58]:
# Bloomingdale
dt = pd.read_excel('new_data_v2.xlsx')
print(dt.shape)

(4526, 26)


In [59]:
#df['selftext'] = df['selftext'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dt.shape

(4526, 26)

# Extracting Material
0. I will make two columns for materials, one is 'material' (main material) and one is 'gem'. 
1. First, extract information from Prod_name because the materials in the prod_name tend to be the main materials. 
2. For the unfilled data in (1) above, I will look at the color which has main colors that could indicate what the main material is.
[ More details are in the comments below]


In [60]:
import re

# Beaded,(268),Fabric,(70),Leather,(33),Mixed Metal,(161),Rose Gold,(415),Silver,(2099),Stainless Steel,(20),White Gold,(35),
# Yellow Gold,(2393), 

# Scraped a Gemstone list from a website
# https://www.crystal-life.com/crystal-gemstone-dictionary/
# response.xpath('//li[@class="stone-list"]/div[@class="index"]/h4/a/text()').extract()

gemstones_lst = ['Abalone Shell',
 'Agate',
 'Agate, Apricot',
 'Agate, Banded',
 'Agate, Black',
 'Agate, Blue Lace',
 'Agate, Botswana',
 'Agate, Fire',
 'Agate, Holly Blue',
 'Agate, Lace',
 'Agate, Lake Superior',
 'Agate, Montana',
 'Agate, Moss',
 'Agate, Orange',
 'Agate, Plume',
 'Agate, Tree',
 'Agate, Turritella',
 'Agate, White',
 'Amazonite',
 'Amber',
 'Ambronite',
 'Amethyst',
 'Amethyst – Fairy',
 'Amethyst: A Versatile Stone',
 'Ametrine',
 'Ammonite',
 'Ammonite, Iridescent',
 'Angelite',
 'Apatite, Blue',
 'Apophyllite',
 'Aquamarine',
 'Aragonite',
 'Atacamite',
 'Atlantisite',
 'Auralite Amethyst',
 'Aventurine, Blue',
 'Aventurine, Green',
 'Aventurine, Red',
 'Azurite',
 'Bauxite',
 'Beryl, Blue',
 'Beryl, Green',
 'Bloodstone',
 'Blue Aventurine',
 'Boji Stones',
 'Brachiopod',
 'Bronzite',
 'Calcite, Angel Wing',
 'Calcite, Blue',
 'Calcite, Cobalto',
 'Calcite, Green',
 'Calcite, Honey',
 'Calcite, Mangano',
 'Calcite, Orange',
 'Calcite, Red',
 'Calcite, White',
 'Calcite, Yellow',
 'Carnelian',
 'Cat’s Eye',
 'Cavansite',
 'Celestite',
 'Chakra (Dynamic)',
 'Chakra (Psychic)',
 'Charoite',
 'Chiastolite “Cross Stone”',
 'Chrysanthemum Stone',
 'Chrysocolla',
 'Chrysoprase',
 'Cinnabarite',
 'Citrine',
 'Copal',
 'Copper',
 'Coral, Black',
 'Coral, Blue',
 'Coral, Red',
 'Coral, Star',
 'Cuprite',
 'Danburite',
 'Desert Rose – Barite',
 'Desert Rose – Selenite',
 'Dinosaur Bone',
 'Diopside',
 'Dragon Stone',
 'Dragonblood',
 'Dumortierite',
 'Earthkeeper Crystals',
 'Emerald',
 'Epidote',
 'Erythrite',
 'Eudialyte',
 'Fairy Stone',
 'Feldspar, Golden',
 'Fluorite, Green',
 'Fluorite, Purple',
 'Fluorite, Rainbow',
 'Fossil Fern – Mazon Creek, IL',
 'Fossil Fern – St. Clair, PA',
 'Fuschite',
 'Galaxyite',
 'Galena',
 'Garnet, Green',
 'Garnet, Red',
 'Geodes',
 'Goldstone',
 'Halite',
 'Hematite',
 'Hematite in Quartz',
 'Hematite, Specular',
 'Hemimorphite',
 'Herkimer Diamond',
 'Hiddenite',
 'Howlite',
 'Hypersthene',
 'Infinite',
 'Iolite',
 'Jade',
 'Jade, African',
 'Jade, Lemurian',
 'Jade, Nephrite',
 'Jade, Serpentine (New Jade)',
 'Jade, Siberian',
 'Jadeite',
 'Jasper',
 'Jasper, Bird’s Eye',
 'Jasper, Caramel',
 'Jasper, Dalmation',
 'Jasper, Impression',
 'Jasper, Kambaba',
 'Jasper, Mookite',
 'Jasper, Ocean',
 'Jasper, Picture',
 'Jasper, Red',
 'Jasper, Silver Leaf',
 'Jasper, Snakeskin',
 'Jasper, Spiderweb',
 'Jasper, Yellow',
 'Jet',
 'K2',
 'Kunzite',
 'Kyanite, Black',
 'Kyanite, Blue',
 'Kyanite, Green',
 'Labradorite',
 'Lapis Lazuli',
 'Larimar',
 'Lava',
 'Leopardskin',
 'Lepidolite',
 'Lionskin',
 'Lodestone',
 'Magnesite',
 'Magnetite',
 'Malachite',
 'Marble',
 'Meteorite',
 'Moldavite',
 'Moonstone',
 'Morganite',
 'Mother of Pearl',
 'Mugglestone',
 'Muscovite',
 'Mystic Merlinite',
 'Nuummite',
 'Obsidian, Apache Tear',
 'Obsidian, Black',
 'Obsidian, Snowflake',
 'Onyx, Black',
 'Onyx, White',
 'Opal',
 'Opal, Blue Peruvian',
 'Opal, Dark',
 'Opal, Pink Peruvian',
 'Orthocerous',
 'Pearl',
 'Peridot',
 'Petoskey Stone',
 'Petrified Wood',
 'Picasso Stone',
 'Pietersite',
 'Portal Stones',
 'Prehnite',
 'Presili Bluestone',
 'Pyrite',
 'Quartz',
 'Quartz – Enhydro',
 'Quartz with Chlorite',
 'Quartz, Aqua Aura',
 'Quartz, Double-Terminated',
 'Quartz, Elestial',
 'Quartz, Faden',
 'Quartz, Lasers & Wands',
 'Quartz, Left and Right Hand',
 'Quartz, Library',
 'Quartz, Needle',
 'Quartz, Quantum',
 'Quartz, Rainbow',
 'Quartz, Rose',
 'Quartz, Rutilated',
 'Quartz, Smokey',
 'Quartz, Tabby',
 'Quartz, Tibetan',
 'Quartz, Tourmalinated',
 'Quartz, with Hematite',
 'Rhodochrosite',
 'Rhodonite',
 'Rhyolite',
 'Ruby',
 'Ruby Fuschsite',
 'Ruby Zoisite',
 'Rudraksha Seed',
 'Sandstone, Dendritic',
 'Sandstone, Kanab',
 'Scolecite',
 'Selenite',
 'Seraphinite',
 'Serpentine',
 'Shattuckite',
 'Shell',
 'Shiva Lingam',
 'Shungite',
 'Sodalite',
 'Stichtite',
 'Stilbite',
 'Stromatolite',
 'Sugilite',
 'Sulphur',
 'Sunstone',
 'Super Seven',
 'Tanzanite',
 'Tektites',
 'Tiger Eye',
 'Tiger Eye, Blue (Hawkeye)',
 'Tiger Eye, Red',
 'Topaz, Blue',
 'Topaz, Imperial',
 'Tourmaline, Black',
 'Tourmaline, Green',
 'Tourmaline, Pink',
 'Tourmaline, Rainbow',
 'Tourmaline, Red',
 'Tourmaline, Watermelon',
 'Turquoise',
 'Turquoise, White',
 'Unakite',
 'Variscite',
 'Vesuvianite',
 'Zebra Stone',
 'Zeolites']

gemstones_lst = [gem.lower() for gem in gemstones_lst if gem.find(",") < 0]
#gemstones = re.compile("|".join(gemstones_lst))
#gemstones_lst.extend(['cubic zirconia', 'crystal', 'diamond', 'mother-of-pearl', 'mother of pearl', 'moonstone', 'pearl', 'amethyst', 'opal', 'turquoise', 'lapis lazuli', 'lapis'])
gems = re.compile("|".join(['cubic zirconia', 'crystal', 'diamond', 'mother-of-pearl', 'mother of pearl', 'moonstone', 'pearl', 'amethyst', 'opal', 'turquoise', 'lapis lazuli', 'lapis', 'onyx', 'pyrite', 'labradorite', 'Hematite', 'glass', 'agate']))
#[pd.Series(desc).apply(lambda x: item_type.findall(str(x))) for desc in dt['desc']]
#a = item_type.findall(str(dt['desc'][0]))



materials_lst = []
metal_lst = ['yellow gold', 'white gold', 'rose gold', 'sterling silver', 'brass','14k gold','18k gold', 'k gold', 'stainless steel', 'rhodium', 'oxidized silver', 'ruthenium']
# Metal other than gold and silver
yellowgold_lst = ['yellow gold', '14k gold','18k gold', 'k gold']
other_metal_lst = ['brass', 'stainless steel', 'rhodium', 'oxidized silver', 'ruthenium']
leather_lst = ['leather']
fabric_lst = ['polyester', 'cotton', 'silk', 'feather', 'rayon', 'nylon']
plastic_lst = ['enamel', 'acetate', 'plastic', 'acrylic', 'lucite', 'resin']

materials_lst.extend(metal_lst)
materials_lst.extend(leather_lst)
materials_lst.extend(fabric_lst)
materials_lst.extend(plastic_lst)

nonmetal_lst = []
nonmetal_lst.extend(leather_lst)
nonmetal_lst.extend(fabric_lst)
nonmetal_lst.extend(plastic_lst)


materials = re.compile("|".join(materials_lst))
metals = re.compile("|".join(metal_lst))
nonmetals = re.compile("|".join(nonmetal_lst))

In [52]:
tempdt = dt.copy()

In [220]:
dt = tempdt

In [61]:
dt['prod_name'] = dt['prod_name'].map(lambda x: x.lower())
dt['desc_list'] = dt['desc_list'].map(lambda x: x.lower())


# 1. Extract info from prod_name
dt['prod_name'] = dt['prod_name'].map(lambda x: x.lower())

dt['main_material'] = dt['prod_name'].map(lambda x: materials.findall(str(x.lower()))[0] if len(materials.findall(str(x.lower())))!=0 else "")
dt['main_material'].loc[(dt.main_material == "") & (dt.prod_name.str.contains("bead"))] = "beads"

dt['main_gem'] = dt['prod_name'].map(lambda x: gems.findall(str(x.lower()))[0] if len(gems.findall(str(x.lower())))!=0 else "")


C:\Users\Soomin\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [62]:
## For Gems

#2. For the unfilled data from above, if there's only one gem matched to a gem in the gemstones list, then it's the main gem.
temp = pd.DataFrame(dt[dt.main_gem == ""]['desc_list'].map(lambda x: gems.findall(str(x.lower()))[0] if len(gems.findall(str(x.lower())))>=1 else ""))
temp.columns = ['main_gem']
dt.update(temp)

#temp = pd.DataFrame(dt[dt.main_gem == ""]['desc_list'].map(lambda x: gems.findall(str(x.lower()))[0] if len(gems.findall(str(x.lower())))==1 else ""))
#temp.columns = ['main_gem']
#dt.update(temp)

In [63]:
## For Materials
#2. For the unfilled data from above, if the description contains "sterling silver" and the item has 'silver' color, 
    # then the main material would be sterling silver.

dt['main_material'].loc[(dt.main_material == "") & \
                       (dt.desc_list.str.contains("k gold") | dt.desc_list.str.contains("k yellow gold") | dt.desc_list.str.contains("kgold")) & \
                        (dt.curr_color.str.match("Gold") | dt.curr_color.str.contains("Gold/"))] = "yellow gold"
dt['main_material'].loc[(dt.main_material == "") & \
                       (dt.desc_list.str.contains("k gold") | dt.desc_list.str.contains("k rose gold") | dt.desc_list.str.contains("kgold")) & \
                        (dt.curr_color.str.match("Rose Gold") | dt.curr_color.str.contains("Rose Gold/") | dt.curr_color.str.contains("/Rose Gold"))] = "rose gold"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("silver")) & (dt.curr_color.str.match("Silver") | dt.curr_color.str.contains("Silver/", na=False))] = "silver"
#dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("k gold") | dt.desc_list.str.contains("k yellow gold") | dt.desc_list.str.contains("kgold")) & (dt.curr_color.str.match("Gold") | dt.curr_color.str.contains("Gold/")| dt.curr_color.str.contains("/Gold"))] = "gold"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("stainless steel")) & (dt.curr_color.str.match("Silver") | dt.curr_color.str.contains("Silver/"))] = "stainless steel"



#3. For the unfilled data from above, plated  -->'plated metal', 'silver-tome, gold-tone' --> metal. 
    # If 'k gold' is included and color is Gold then it's categorized as 'gold'

#dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("plated") | (dt.desc_list.str.contains("plate")| dt.desc_list.str.contains("vermeil") | dt.desc_list.str.contains("dipped")))] = "plated metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("gold-tone") |  dt.desc_list.str.contains("gold tone") | dt.desc_list.str.contains("goldtone"))\
                       & (dt.curr_color.str.match("Gold") | dt.curr_color.str.contains("Gold/"))] = "metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("silver-tone") |  dt.desc_list.str.contains("silver tone") | dt.desc_list.str.contains("silvertone"))\
                       & (dt.curr_color.str.match("Silver") | dt.curr_color.str.contains("Silver/"))] = "metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("plated") |  dt.desc_list.str.contains("plate") | dt.desc_list.str.contains("vermeil") | dt.desc_list.str.contains("dipped"))\
                       & (dt.curr_color.str.match("Silver") | dt.curr_color.str.contains("Silver/") | dt.curr_color.str.match("Gold") | dt.curr_color.str.contains("Gold/"))] = "metal"

#4. In case of the 'post back' earrings and Color doesn't have 'Multi/Silver/Gold', which means metal is really not its main material 
    # --> main material should not be metal, so put the first material after excluding any material in the metal list
temp = pd.DataFrame(dt[['prod_name', 'desc_list', 'curr_color', 'prod_image', 'main_gem']][(dt.main_material == "") & dt.desc_list.str.contains("post") & dt.desc_list.str.contains("back") & (dt.curr_color.str.contains("Silver|Gold|Multi", case=False, regex=True)==False)]\
['desc_list'].map(lambda x: nonmetals.findall(str(x.lower()))[0] if len(nonmetals.findall(str(x.lower())))>=1 else ""))
temp.columns = ['main_material']
dt.update(temp)

#4. For the 'Multi' color items -->  apply #2 again (and for '*k gold' too)
dt['main_material'].loc[(dt.main_material == "") & \
                       (dt.desc_list.str.contains("k gold") | dt.desc_list.str.contains("k yellow gold") | dt.desc_list.str.contains("kgold")) & \
                        (dt.curr_color.str.contains("Multi"))] = "yellow gold"
dt['main_material'].loc[(dt.main_material == "") & \
                       (dt.desc_list.str.contains("k gold") | dt.desc_list.str.contains("k rose gold") | dt.desc_list.str.contains("kgold")) & \
                        (dt.curr_color.str.contains("Multi"))] = "rose gold"
dt['main_material'].loc[(dt.main_material == "") & dt.desc_list.str.contains("sterling silver") & dt.curr_color.str.contains("Multi", na=False)] = "silver"
#dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("k gold") | dt.desc_list.str.contains("k yellow gold") | dt.desc_list.str.contains("kgold")) & (dt.curr_color.str.match("Gold") | dt.curr_color.str.contains("Gold/")| dt.curr_color.str.contains("/Gold"))] = "gold"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("stainless steel")) & dt.curr_color.str.contains("Multi")] = "stainless steel"

#5. For the 'Multi' color items -->  apply #3 again 
#dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("plated") | (dt.desc_list.str.contains("plate")| dt.desc_list.str.contains("vermeil") | dt.desc_list.str.contains("dipped")))] = "plated metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("gold-tone") |  dt.desc_list.str.contains("gold tone") | dt.desc_list.str.contains("goldtone"))\
                       & dt.curr_color.str.contains("Multi")] = "metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("silver-tone") |  dt.desc_list.str.contains("silver tone") | dt.desc_list.str.contains("silvertone"))\
                       & dt.curr_color.str.contains("Multi")] = "metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("plated") |  dt.desc_list.str.contains("plate") | dt.desc_list.str.contains("vermeil") | dt.desc_list.str.contains("dipped"))\
                       & dt.curr_color.str.contains("Multi")] = "metal"


#6. 색깔에 '/gold', '/silver' 들어가 있으면서 #2,#3 반복
dt['main_material'].loc[(dt.main_material == "") & \
                       (dt.desc_list.str.contains("k gold") | dt.desc_list.str.contains("k yellow gold") | dt.desc_list.str.contains("kgold")) & \
                        dt.curr_color.str.contains("/Gold/")] = "yellow gold"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("silver")) & (dt.curr_color.str.match("Silver") | dt.curr_color.str.contains("/Silver", na=False))] = "silver"
#dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("k gold") | dt.desc_list.str.contains("k yellow gold") | dt.desc_list.str.contains("kgold")) & (dt.curr_color.str.match("Gold") | dt.curr_color.str.contains("Gold/")| dt.curr_color.str.contains("/Gold"))] = "gold"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("stainless steel")) & (dt.curr_color.str.match("Silver") | dt.curr_color.str.contains("/Silver"))] = "stainless steel"



#7. For the unfilled data from above, plated  -->'plated metal', 'silver-tome, gold-tone' --> metal. If 'k gold' is included and color is Gold then it's categorized as 'gold'

#dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("plated") | (dt.desc_list.str.contains("plate")| dt.desc_list.str.contains("vermeil") | dt.desc_list.str.contains("dipped")))] = "plated metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("gold-tone") |  dt.desc_list.str.contains("gold tone") | dt.desc_list.str.contains("goldtone"))\
                       & (dt.curr_color.str.match("Gold") | dt.curr_color.str.contains("/Gold"))] = "metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("silver-tone") |  dt.desc_list.str.contains("silver tone") | dt.desc_list.str.contains("silvertone"))\
                       & (dt.curr_color.str.match("Silver") | dt.curr_color.str.contains("/Silver"))] = "metal"
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("plated") |  dt.desc_list.str.contains("plate") | dt.desc_list.str.contains("vermeil") | dt.desc_list.str.contains("dipped"))\
                       & (dt.curr_color.str.match("Silver") | dt.curr_color.str.contains("/Silver") | dt.curr_color.str.match("Gold") | dt.curr_color.str.contains("/Gold"))] = "metal"

# 8. For all the others (about 441 items)
temp = pd.DataFrame(dt[(dt.main_material == "")]['desc_list'].map(lambda x: materials.findall(str(x.lower()))[0] if len(materials.findall(str(x.lower())))>=1 else ""))
temp.columns = ['main_material']
dt.update(temp)

# 9. For the others --> if it has gold/silver in the color, it's metal
dt['main_material'].loc[(dt.main_material == "") & dt.curr_color.str.contains("Silver|Gold|Rhodium|metal", case=False, regex=True)] = "metal"

# 10. When the main gem is filled, material list에서 desc에 들어가 있는 젤 처음꺼를 main material로.. (0 items..)
temp = pd.DataFrame(dt[(dt.main_material == "") & (dt.main_gem != "")]['desc_list'].map(lambda x: materials.findall(str(x.lower()))[0] if len(materials.findall(str(x.lower())))>=1 else ""))
temp.columns = ['main_material']
dt.update(temp)

# 11. When the main gem is filled, if the desc has gold/silver/metal list (gold tone, silver tone, plated) ,then the main material is metal.
dt['main_material'].loc[(dt.main_material == "") & (dt.main_gem != "") & (dt.desc_list.str.contains("gold-tone|gold tone|goldtone|silver-tone|silver tone|silvertone|plated|plate|vermeil|dipped", case=False, regex=True))] = "metal"


# --> Reduced to 15 items


# 12. Final step: Convert the more detailed Materials to general materials
def general_material_cat(x):
    if "silver" in x: 
        return "silver"
    elif "stainless steel" in x:
        return "stainless steel"
    elif x in yellowgold_lst:
        return "yellow gold"
    elif x in fabric_lst:
        return "fabric"
    elif x in plastic_lst:
        return "plastic"
    elif x in other_metal_lst:
        return "metal"
    else:
        return x

dt['main_material'] = dt['main_material'].map(general_material_cat)


C:\Users\Soomin\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [64]:
dt.main_material.unique()

array(['beads', 'silver', 'yellow gold', 'rose gold', 'white gold',
       'metal', 'plastic', 'fabric', 'stainless steel', 'leather', ''],
      dtype=object)

In [56]:
dt

,page_num,product_num,prod_name,brand,curr_color,price_dis,price_reg,color_by_price,desc_list,related_search,full_file_name,prod_image,main_material,main_gem
0,1,1,"station chain necklace, 36""",Crislu,Gold,$128.00,$128.00,{'$128.00': ['Gold']},"36""l with 2"" extension,lobster clasp closure,18k gold-plated sterling silver/cubic zirconia,hypoallergenic,manufacturer's lifetime warranty,photo may have been enlarged and/or enhanced,imported,,,web id: 1454115","Majorca Pearls,Majorica Pearls,Pearl Drop Earrings,Pearl Ear Rings,Pearl Earrings,White Pearl",img1033386_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/6/optimized/1033386_fpx.tif,yellow gold,cubic zirconia
1,1,2,knot & simulated pearl drop earrings,Majorica,Rhodium/White Pearl,$80.00,$80.00,{'$80.00': ['Rhodium/White Pearl']},"1.25"" drop,french wire,10mm organic man-made pearls,sterling silver/organic man-made pearls,photo may have been enlarged and/or enhanced,made in spain,,,web id: 831214",NaN,img8395599_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/9/optimized/8395599_fpx.tif,silver,pearl
2,1,3,amethyst small hoop earrings in 14k yellow gold - 100% exclusive,Bloomingdale's,Purple/Gold,$200.00,$400.00,{'$200.00': ['Purple/Gold']},"14k yellow gold/amethyst,amethyst: .80 ct. t.w.,1""l,post back,almost all gemstones and black diamonds have been treated to enhance their beauty and require special care,photo may have been enlarged and/or enhanced,amethyst is a february birthstone,imported,,,web id: 1126646","Amethyst Earrings,Purple Earrings,Purple Jewelry,Small Earrings,Small Gold Hoop Earrings,Small Hoop Earrings",img8641198_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/8/optimized/8641198_fpx.tif,yellow gold,amethyst
3,1,4,huggie hoop earrings,Robert Lee Morris Soho,Silver,$28.00,$28.00,{'$28.00': ['Silver']},"silver plated; cubic zirconia,0.7"" diameter,0.5"" width,post back,photo may have been enlarged and/or enhanced,imported,web id: 3027108","Huggie Earrings,Huggie Hoop Earrings,Robert Lee Morris Jewelry,Robert Lee Morris Soho,Sterling Silver Hoop Earrings,Sterling Silver Hoops",img9904054_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/4/optimized/9904054_fpx.tif,silver,cubic zirconia
4,1,5,14k yellow gold circle charm bracelet with diamond,Zoë Chicco,White/Gold,$385.00,$385.00,{'$385.00': ['White/Gold']},"diamonds: .05 ct. t.w.,14k yellow gold,6-7"" adjustable length; charm: .5l,spring ring,photo may have been enlarged and/or enhanced,made in usa,,,web id: 2692423","14k Gold Charms,Charm Bracelet Charms,Charm Bracelets,Charm Bracelets For Women,Gold Charm Bracelet,Zoe Chicco",img9584178_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/8/optimized/9584178_fpx.tif,yellow gold,diamond
5,1,6,14k yellow gold diamond mini circle ring,MATEO,White/Gold,$300.00,$300.00,{'$300.00': ['White/Gold']},"diamonds: 0.06 ct. t.w.,14k yellow gold,photo may have been enlarged and/or enhanced,made in usa,,,web id: 2639432","Dana Rebecca,Dana Rebecca Designs,Dana Rebecca Designs Earrings,Dana Rebecca Designs Necklaces,Dana Rebecca Necklace,Gold Fashion Rings",img9792084_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/4/optimized/9792084_fpx.tif,yellow gold,diamond
6,1,7,wavy hoop earrings,Argento Vivo,Gold,$98.00,$98.00,{'$98.00': ['Gold']},"18k gold–plated sterling silver or sterling silver,2.5"" diameter,snap-back closure,photo may have been enlarged and/or enhanced,imported,web id: 2869785","Argento Vivo,Gold And Silver Hoop Earrings,Silver Hoop Earrings,Silver Hoops,Sterling Silver Hoop Earrings,Sterling Silver Hoops",img9775066_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/6/optimized/9775066_fpx.tif,yellow gold,
7,1,8,sterling silver & cubic zirconia abstract elegance jacket earrings,PANDORA,White/Silver,$70.00,$70.00,{'$70.00': ['White/Silver']},"0.6""l,post back,sterling silver; cubic zirconia,photo may have been enlarged and/

In [66]:
dt.to_excel("material_with_newdata_Soomin_Sep20.xlsx")

In [24]:
#dt[['prod_name', 'desc_list', 'curr_color', 'prod_image', 'main_gem']][(dt.main_material == "") & (dt.desc_list.str.contains("post")) & (dt.desc_list.str.contains("back")) & (len(metals.findall(str(dt.desc_list))) >= 0)]

#dt[['prod_name', 'desc_list', 'curr_color', 'prod_image', 'main_gem']][(dt.main_material == "") & (dt.curr_color.str.contains("Gold")) & (len(metals.findall(str(dt.desc_list))) >= 0)]

dt[['prod_name', 'desc_list', 'curr_color', 'prod_image', 'main_gem']][(dt.main_material == "")]


,prod_name,desc_list,curr_color,prod_image,main_gem
160,lobe wonder™ earring support patches,"includes 60 patches,patches attach to the back of the earlobe allowing you to wear big chandelier earrings for hours,photo may have been enlarged and/or enhanced,imported,web id: 1572104",White,https://images.bloomingdalesassets.com/is/image/BLM/products/1/optimized/9739291_fpx.tif,
582,two tone orbital earrings,"2"" drop,french wire,photo may have been enlarged and/or enhanced,silver-tone and gold-tone plated metal,imported,web id: 1176548",Two Tone,https://images.bloomingdalesassets.com/is/image/BLM/products/1/optimized/8676941_fpx.tif,
708,"women's 8mm round white simulated pearl endless necklace, 60""","organic man made pearls,imported,web id: 127948",White,https://images.bloomingdalesassets.com/is/image/BLM/products/2/optimized/1113472_fpx.tif,pearl
930,women's 11mm round simulated pearl stud earrings,"organic man made pearls with 14 karat post,,,web id: 127942",Gray,https://images.bloomingdalesassets.com/is/image/BLM/products/0/optimized/1113470_fpx.tif,pearl
1330,"strand necklace, 72""","72""l,no closure,photo may have been enlarged and/or enhanced,glass,imported,web id: 1212731",White,https://images.bloomingdalesassets.com/is/image/BLM/products/1/optimized/1206751_fpx.tif,glass
1349,two tone clip-on hoop earrings,"0.7"" drop,clip-on backs,silver-tone and gold-tone plated metal,photo may have been enlarged and/or enhanced,imported,web id: 1785033",Two Tone,https://images.bloomingdalesassets.com/is/image/BLM/products/6/optimized/9208216_fpx.tif,
1433,thomas geo chandelier earrings,"painted metal; glass or amethyst,2.75""l,french wire,photo may have been enlarged and/or enhanced,imported,web id: 3013785",Matte Lilac,https://images.bloomingdalesassets.com/is/image/BLM/products/2/optimized/9899652_fpx.tif,glass
2092,"elisa matte necklace, 15""","painted metal; mother-of-pearl or howlite,15""l with 2"" extension,0.3""l pendant,lobster clasp closure,photo may have been enlarged and/or enhanced,imported,web id: 2956868",Blue,https://images.bloomingdalesassets.com/is/image/BLM/products/6/optimized/9834956_fpx.tif,mother-of-pearl
2277,floral stud earrings,"0.39"" diameter,post backs,photo may have been enlarged and/or enhanced,crystal/surgical steel posts,imported,web id: 1024361",Clear,https://images.bloomingdalesassets.com/is/image/BLM/products/7/optimized/8516587_fpx.tif,crystal
2754,crest bangle bracelet,"gold-tone metal; epoxy,6"" inner circumference,no closure,photo may have been enlarged and/or enhanced,imported,web id: 2991601",Burgundy,https://images.bloomingdalesassets.com/is/image/BLM/products/5/optimized/9972325_fpx.tif,


In [65]:
dt[(dt.main_material == "")].shape
#dt[(dt.main_material == "")].shape

(15, 28)

In [67]:
pd.DataFrame(dt[(dt.main_material == "") & (dt.main_gem == "") & dt.curr_color.str.contains("Silver|Gold|Rhodium|metal", case=False, regex=True)]['desc_list'])

,desc_list
282,"this necklace can be customized with alex and ani necklace charms (sold separately).,web id: 1760439"
448,"2.5"" drop,french wire,gold tone–plated metal,photo may have been enlarged and/or enhanced,imported,web id: 1759245"
776,"gold or silver-tone metal,7"" inner circumference,hinge closure,photo may have been enlarged and/or enhanced,imported,web id: 2981351"
949,"gunmetal-plated metal; black drusy,15""l with 2"" extension,0.41"" width pendant,lobster clasp closure,photo may have been enlarged and/or enhanced,imported,web id: 3064555"
1057,"style #387250-60,23.6""l,14k rose-gold plated,pandora rose collection,photo may have been enlarged and/or enhanced,imported,,,web id: 2898341"
1203,"18""l with 2"" extension,lobster clasp closure,2""l pendant,silver tone–plated metal,photo may have been enlarged and/or enhanced,imported,web id: 1605206"
1851,web id: 1296255
3277,"hypoallergenic,manufacturer's lifetime warranty,,,web id: 1376116"
3323,web id: 1165677
3654,web id: 514033


In [58]:
pd.DataFrame(dt[dt.main_gem == ""]['desc_list'].map(lambda x: gems.findall(str(x.lower())) if len(gems.findall(str(x.lower())))>=1 else ""))

,desc_list
6,
10,
12,
13,"[cubic zirconia, glass]"
18,"[diamond, diamond, diamond]"
19,"[diamond, diamond, diamond]"
21,
22,
23,
24,


In [37]:
#3. For the unfilled data from above, if there's only one material matched to a material 
   # in the material list, then it's the main material.

temp = pd.DataFrame(dt[(dt.main_material == "")]['desc_list'].map(lambda x: materials.findall(str(x.lower()))[0] if len(materials.findall(str(x.lower())))==1 else ""))
temp.columns = ['main_material']
dt.update(temp)


C:\Users\Soomin\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [38]:
#4. For the unfilled data from above, if the description contains "metal", then the main material would be metal.
dt['main_material'].loc[(dt.main_material == "") & (dt.desc_list.str.contains("metal"))] = "metal"


#6. 
# So far, the remaining empty main_materials fields are 22. These fields will be manually filled.



C:\Users\Soomin\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [49]:
# Checking the rows whose main_material field is not filled yet.
dt[(dt.main_material == "")]

,page_num,product_num,prod_name,brand,curr_color,price_dis,price_reg,color_by_price,desc_list,related_search,full_file_name,prod_image,main_material,main_gem
1,1,2,knot & simulated pearl drop earrings,Majorica,Rhodium/White Pearl,$80.00,$80.00,{'$80.00': ['Rhodium/White Pearl']},"1.25"" drop,french wire,10mm organic man-made pearls,sterling silver/organic man-made pearls,photo may have been enlarged and/or enhanced,made in spain,,,web id: 831214",NaN,img8395599_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/9/optimized/8395599_fpx.tif,,pearl
3,1,4,huggie hoop earrings,Robert Lee Morris Soho,Silver,$28.00,$28.00,{'$28.00': ['Silver']},"silver plated; cubic zirconia,0.7"" diameter,0.5"" width,post back,photo may have been enlarged and/or enhanced,imported,web id: 3027108","Huggie Earrings,Huggie Hoop Earrings,Robert Lee Morris Jewelry,Robert Lee Morris Soho,Sterling Silver Hoop Earrings,Sterling Silver Hoops",img9904054_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/4/optimized/9904054_fpx.tif,,
9,1,10,mother of pearl double drop earrings,Nadri,Sterling Silver/Mother of Pearl,$195.00,$195.00,{'$195.00': ['Sterling Silver/Mother of Pearl']},"1.25"" drop,leverback,photo may have been enlarged and/or enhanced,rhodium-plated sterling silver/cubic zirconia/mother of pearl,imported,web id: 1144673","Nadri Earrings,Nadri Jewelry,Nardi Earrings,Sterling Silver,Sterling Silver Earrings,Sterling Silver Jewelry",img8655793_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/3/optimized/8655793_fpx.tif,,mother of pearl
12,1,13,oval hoop earrings,Lauren Ralph Lauren,Silver,$32.00,$32.00,{'$32.00': ['Silver']},"1.4"" drop,leverback,silver tone–plated metal,photo may have been enlarged and/or enhanced,imported,web id: 1586794","Lauren B Jewelry,Ralph Lauren Earrings,Ralph Lauren Jewelry,Ralph Lauren Lauren Jewelry,Silver Hoop Earrings,Silver Hoops",img8975713_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/3/optimized/8975713_fpx.tif,,
13,1,14,small hoop drop earrings,Nadri,Silver,$35.00,$35.00,{'$35.00': ['Silver']},"0.75"" drop,post backs,rhodium–plated brass/cubic zirconia/glass,photo may have been enlarged and/or enhanced,imported,web id: 2806259","Dangle Earrings,Nadri Earrings,Nadri Jewelry,Silver Drop Earrings,Small Hoop Earrings,Small Silver Hoop Earrings",img9716532_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/2/optimized/9716532_fpx.tif,,
14,1,15,elysian hoop earrings,BAUBLEBAR,Multi,$42.00,$42.00,{'$42.00': ['Multi']},"brass; acrylic & glass stones; polyester,1.45"" drop,post back,photo may have been enlarged and/or enhanced,imported,web id: 2981523","Babble Bar,Baublebar Earrings,Baublebar Jewelry,Designer Earrings,Gold Hoop Earrings,Gold Hoops",img9859677_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/7/optimized/9859677_fpx.tif,,
18,1,19,"ivy double wrap gia necklace, 21.5"" - 100% exclusive",Jennifer Zeuner,Black,$32.20,$230.00,{'$32.20': ['Black']},"21.5""l with 2.5"" extension,lobster clasp closure,diamond: 0.15 ct. t.w.,this item is part of our 100% collection, featuring exclusive capsules and unique designs you won’t find anywhere else.,leather/sterling silver/genuine diamond,almost all gemstones and black diamonds have been treated to enhance their beauty and require special care,photo may have been enlarged and/or enhanced,made in usa,web id: 1730076","Fashion Jewelry,Jennifer Meyer Jewelry,Jennifer Zeuner,Necklaces For Women,Women's Fashion,Y Necklaces",img9188694_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/4/optimized/9188694_fpx.tif,,
21,1,22,dxb dubai united arab emirates luggage tag charm,Jet Set Candy,Silver,$68.00,$68.00,{'$68.00': ['Silver']},"0.87""l,no closure,chain sold separately,14k gold vermeil or sterling silver,this charm can be added to any chain necklace or jet set candy's infinity link bracelet.,can be paired with,photo may have been enlarged and/or enha

In [29]:
dt.main_material.iloc[31] = "leather"

C:\Users\Soomin\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
pd.DataFrame(dt[(dt.main_material == "")]['desc_list'].map(lambda x: materials.findall(str(x.lower())) if len(materials.findall(str(x.lower())))>0 else ""))

,desc_list
14,"[brass, acrylic, glass, polyester]"
18,"[leather, sterling silver]"
31,"[white gold, stainless steel, stainless steel]"
160,
167,"[acrylic, polyester, steel]"
195,"[brass, glass]"
282,
285,"[polyester, cotton, glass]"
324,"[plastic, polyester]"
456,"[brass, glass]"


# The aws comprehend - Keyphrases

In [24]:
import boto3
import json
import bson
#from bson import json_util
 
comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')
                
input_s3_url = "s3://aws-logs-674615585406-us-east-1/description_data.xlsx"
input_doc_format = "ONE_DOC_PER_LINE"
output_s3_url = "s3://aws-logs-674615585406-us-east-1/result"
data_access_role_arn = "arn:aws:iam::674615585406:role/comprehend"
number_of_topics = 10
 
input_data_config = {"S3Uri": "s3://aws-logs-674615585406-us-east-1/description_data.xlsx", "InputFormat": "ONE_DOC_PER_LINE"}
output_data_config = {"S3Uri": "s3://aws-logs-674615585406-us-east-1/result"}
 
start_topics_detection_job_result = comprehend.start_topics_detection_job(NumberOfTopics=number_of_topics,
                                                                              InputDataConfig=input_data_config,
                                                                              OutputDataConfig=output_data_config,
                                                                              DataAccessRoleArn=data_access_role_arn)
 
print('start_topics_detection_job_result: ' + json.dumps(start_topics_detection_job_result))
 
job_id = start_topics_detection_job_result["JobId"]
 
print('job_id: ' + job_id)
 
describe_topics_detection_job_result = comprehend.describe_topics_detection_job(JobId=job_id)
 
print('describe_topics_detection_job_result: ' + json.dumps(describe_topics_detection_job_result, default=json_util.default))
 
list_topics_detection_jobs_result = comprehend.list_topics_detection_jobs()
 
print('list_topics_detection_jobs_result: ' + json.dumps(list_topics_detection_jobs_result, default=json_util.default))

ClientError: An error occurred (AccessDeniedException) when calling the StartTopicsDetectionJob operation: User: arn:aws:iam::674615585406:user/juliepark is not authorized to perform: comprehend:StartTopicsDetectionJob

In [177]:
comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')
indices = []
prod_name = []
key_phrases = []
score = []

#for i in xrange(0, dt.shape[0]):
#for desc in dt['desc']:
#    result_dict = comprehend.detect_key_phrases(Text=desc, LanguageCode='en')
#    for d in result_dict['KeyPhrases']:
#        index = desc.index
#        key_phrases.append(d['Text'])
#        score.append(d['Score'])

for index, row in dt.iterrows():
    result_dict = comprehend.detect_key_phrases(Text=row.desc, LanguageCode='en')
    for d in result_dict['KeyPhrases']:
        indices.append(index)
        prod_name.append(row.prod_name)
        key_phrases.append(d['Text'])
        score.append(d['Score'])

In [202]:
key_phrases

['Station Chain Necklace',
 '36',
 '36"L',
 '2" extension',
 'Lobster clasp closure',
 '18K gold-plated sterling silver',
 'cubic zirconia',
 'Hypoallergenic',
 'Manufacturer',
 'lifetime warranty',
 'Imported',
 'Knot & Simulated Pearl Drop Earrings',
 '1.25" drop',
 'French wire',
 '10mm organic man-made pearls',
 'Sterling silver/organic man-made pearls',
 'Spain',
 'Amethyst Small Hoop Earrings',
 '14K Yellow Gold',
 '100%',
 '14k yellow gold/amethyst',
 'Amethyst',
 '80 ct',
 't.w.',
 '1"L',
 'Post',
 'Almost all gemstones',
 'black diamonds',
 'their beauty',
 'special care',
 'Amethyst',
 'a February birthstone',
 'Imported',
 'Huggie Hoop Earrings',
 'Silver',
 'cubic zirconia',
 '0.7" diameter',
 '0.5" width',
 'Post',
 '14K Yellow Gold Circle Charm Bracelet',
 'Diamond,Diamonds',
 '05 ct',
 't.w.',
 '14K yellow gold',
 '6-7" adjustable length',
 'charm',
 '5\x94L',
 'Spring ring',
 'USA',
 '14K Yellow Gold Diamond Mini Circle Ring',
 'Diamonds',
 '0.06 ct',
 't.w.',
 '14K yel

In [215]:
kp_result = pd.DataFrame({'index': indices, 'prod_name': prod_name, 'score': score, 'key_phrase':key_phrases})
#kp_result.columns = ['index', 'prod_name', 'key_phrase', 'score']
kp_result

,index,prod_name,score,key_phrase
0,0,"Station Chain Necklace, 36""",0.974409,Station Chain Necklace
1,0,"Station Chain Necklace, 36""",0.663907,36
2,0,"Station Chain Necklace, 36""",0.865901,"36""L"
3,0,"Station Chain Necklace, 36""",0.922661,"2"" extension"
4,0,"Station Chain Necklace, 36""",0.855653,Lobster clasp closure
5,0,"Station Chain Necklace, 36""",0.906810,18K gold-plated sterling silver
6,0,"Station Chain Necklace, 36""",0.870458,cubic zirconia
7,0,"Station Chain Necklace, 36""",0.847808,Hypoallergenic
8,0,"Station Chain Necklace, 36""",0.998265,Manufacturer
9,0,"Station Chain Necklace, 36""",0.977322,lifetime warranty


In [248]:
dt.head()

,page_num,product_num,prod_name,brand,curr_color,price_dis,price_reg,color_by_price,desc_list,related_search,full_file_name,prod_image,desc,type
0,1,1,"Station Chain Necklace, 36""",Crislu,Gold,$128.00,$128.00,{'$128.00': ['Gold']},"36""L with 2"" extension,Lobster clasp closure,18K gold-plated sterling silver/cubic zirconia,Hypoallergenic,Manufacturer's lifetime warranty,Photo may have been enlarged and/or enhanced,Imported,,,Web ID: 1454115","Majorca Pearls,Majorica Pearls,Pearl Drop Earrings,Pearl Ear Rings,Pearl Earrings,White Pearl",img1033386_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/6/optimized/1033386_fpx.tif,"Station Chain Necklace, 36"",36""L with 2"" extension,Lobster clasp closure,18K gold-plated sterling silver/cubic zirconia,Hypoallergenic,Manufacturer's lifetime warranty,Imported,,",Chain
1,1,2,Knot & Simulated Pearl Drop Earrings,Majorica,Rhodium/White Pearl,$80.00,$80.00,{'$80.00': ['Rhodium/White Pearl']},"1.25"" drop,French wire,10mm organic man-made pearls,Sterling silver/organic man-made pearls,Photo may have been enlarged and/or enhanced,Made in Spain,,,Web ID: 831214",NaN,img8395599_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/9/optimized/8395599_fpx.tif,"Knot & Simulated Pearl Drop Earrings,1.25"" drop,French wire,10mm organic man-made pearls,Sterling silver/organic man-made pearls,Made in Spain,,",Earrings
2,1,3,Amethyst Small Hoop Earrings in 14K Yellow Gold - 100% Exclusive,Bloomingdale's,Purple/Gold,$200.00,$400.00,{'$200.00': ['Purple/Gold']},"14k yellow gold/amethyst,Amethyst: .80 ct. t.w.,1""L,Post back,Almost all gemstones and black diamonds have been treated to enhance their beauty and require special care,Photo may have been enlarged and/or enhanced,Amethyst is a February birthstone,Imported,,,Web ID: 1126646","Amethyst Earrings,Purple Earrings,Purple Jewelry,Small Earrings,Small Gold Hoop Earrings,Small Hoop Earrings",img8641198_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/8/optimized/8641198_fpx.tif,"Amethyst Small Hoop Earrings in 14K Yellow Gold - 100% Exclusive,14k yellow gold/amethyst,Amethyst: .80 ct. t.w.,1""L,Post back,Almost all gemstones and black diamonds have been treated to enhance their beauty and require special care,Amethyst is a February birthstone,Imported,,",Earrings
3,1,4,Huggie Hoop Earrings,Robert Lee Morris Soho,Silver,$28.00,$28.00,{'$28.00': ['Silver']},"Silver plated; cubic zirconia,0.7"" diameter,0.5"" width,Post back,Photo may have been enlarged and/or enhanced,Imported,Web ID: 3027108","Huggie Earrings,Huggie Hoop Earrings,Robert Lee Morris Jewelry,Robert Lee Morris Soho,Sterling Silver Hoop Earrings,Sterling Silver Hoops",img9904054_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/4/optimized/9904054_fpx.tif,"Huggie Hoop Earrings,Silver plated; cubic zirconia,0.7"" diameter,0.5"" width,Post back,Imported",Earrings
4,1,5,14K Yellow Gold Circle Charm Bracelet with Diamond,Zoë Chicco,White/Gold,$385.00,$385.00,{'$385.00': ['White/Gold']},"Diamonds: .05 ct. t.w.,14K yellow gold,6-7"" adjustable length; charm: .5L,Spring ring,Photo may have been enlarged and/or enhanced,Made in USA,,,Web ID: 2692423","14k Gold Charms,Charm Bracelet Charms,Charm Bracelets,Charm Bracelets For Women,Gold Charm Bracelet,Zoe Chicco",img9584178_fpx_tif,https://images.bloomingdalesassets.com/is/image/BLM/products/8/optimized/9584178_fpx.tif,"14K Yellow Gold Circle Charm Bracelet with Diamond,Diamonds: .05 ct. t.w.,14K yellow gold,6-7"" adjustable length; charm: .5L,Spring ring,Made in USA,,",Charm


In [9]:
# DetectKeyPhrases
comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')
                
text = '14K gold plated, 14K rose gold plated or rhodium plated; jade, mother-of-pearl, rhodonite or turquoise,17"L with 2" extension,5.5" lariat drop,Lobster clasp closure,Photo may have been enlarged and/or enhanced.,Imported,Web ID: 3082689'

print('Calling DetectKeyPhrases')
#print(json.dumps(comprehend.detect_key_phrases(Text=text, LanguageCode='en'), sort_keys=True, indent=4))
result_json = json.dumps(comprehend.detect_key_phrases(Text=text, LanguageCode='en'), sort_keys=True, indent=4)
result_dic = comprehend.detect_key_phrases(Text=text, LanguageCode='en')
print('End of DetectKeyPhrases\n')

Calling DetectKeyPhrases
End of DetectKeyPhrases



In [14]:
# Detecting syntax
comprehend = boto3.client(service_name = 'comprehend', region_name='us-east-1')
text = "It is raining today in Seattle"
 
print('Calling DetectSyntax')
print(json.dumps(comprehend.detect_syntax(Text=text, LanguageCode='en'), sort_keys=True, indent=4))
print('End of DetectSyntax\n')

Calling DetectSyntax
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "715",
            "content-type": "application/x-amz-json-1.1",
            "date": "Tue, 11 Sep 2018 23:04:59 GMT",
            "x-amzn-requestid": "1b46124a-b617-11e8-b6aa-1188a630efab"
        },
        "HTTPStatusCode": 200,
        "RequestId": "1b46124a-b617-11e8-b6aa-1188a630efab",
        "RetryAttempts": 0
    },
    "SyntaxTokens": [
        {
            "BeginOffset": 0,
            "EndOffset": 2,
            "PartOfSpeech": {
                "Score": 0.9999730587005615,
                "Tag": "PRON"
            },
            "Text": "It",
            "TokenId": 1
        },
        {
            "BeginOffset": 3,
            "EndOffset": 5,
            "PartOfSpeech": {
                "Score": 0.9338240027427673,
                "Tag": "AUX"
            },
            "Text": "is",
            "TokenId": 2
        },
        {